In [ ]:
import csv
import time
import pandas as pd
import random
import utils
import os
import multiprocessing
from functools import partial
from classes.gift import Gift
from classes.trip import Trip

In [ ]:
random.seed(50)

DATA_PATH = "./data/gifts.csv"
SLEIGH_CAPACITY = 1000
INITIAL_POPULATION_SIZE = 50000
MAX_ITERATIONS = 10000
TOP_K = 10

## Read data into list

In [ ]:
start = time.time()
print("Initiating Dataset Loading...")

df = pd.read_csv(DATA_PATH)
gift_list = []

for i in range(len(df)):
    gift_series = df.iloc[i]
    gift = Gift(gift_series.GiftId, gift_series.Latitude, gift_series.Longitude, gift_series.Weight)
    gift_list.append(gift)
    
end = time.time()
print("Time Taken to load dataset:", end-start)

## Generate Initial population

Create initial population output folder

In [ ]:
dir_path = "./initial_population/" + str(int(time.time()))
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [ ]:
def generate_population(initial_population, i):
    if (i+1) % 100 == 0:
        print("generating population", i+1)
        
    file_path = os.path.join(dir_path, "trip-{0:0=5d}.csv".format(i+1))
    
    with open(file_path, mode="w") as trip_file:
        trip_writer = csv.writer(trip_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        trip_writer.writerow(["tripId", "giftId", "latitude", "longitude", "weight"])
        random.shuffle(gift_list)
        total_weight = 0
        trip_list = []

        j = 0
        trip_id = 1
        while j < len(gift_list):
            gift_trip_list = []
            total_weight = 0
            while j < len(gift_list) and (total_weight + gift_list[i].weight) <= SLEIGH_CAPACITY:
                curr_gift = gift_list[j]
                gift_trip_list.append(curr_gift)
                total_weight = total_weight + curr_gift.weight
                trip_writer.writerow([trip_id, curr_gift.giftId, curr_gift.latitude, curr_gift.longitude, curr_gift.weight])
                j += 1

            trip_id += 1
            trip_order = Trip(gift_trip_list, utils.tripCost(gift_trip_list))
            trip_list.append(trip_order)

    initial_population.append(trip_list)
    

In [ ]:
start = time.time()
print("Initiating population...")

pool = multiprocessing.Pool(processes = multiprocessing.cpu_count()-1)
initial_population = multiprocessing.Manager().list()
size_list = list(range(INITIAL_POPULATION_SIZE))
func = partial(generate_population, initial_population)

pool.map(func, size_list)
pool.close()
pool.join()

end = time.time()
print("Time taken to init population:", end-start)